In [ ]:
!pip3 install box2d-py
!pip3 install gym[Box_2D]

In [ ]:
import gym

env = gym.make('LunarLander-v2')

n_games = 100

for i in range(n_games):
    obs_nul = env.reset()
    score = 0
    done = False
    while not done:
        action = env.action_space.sample()
        obs_prime, reward, done, info = env.step(action)
        score += reward
        # env.render()
    print("Episode: {}\t Reward:{}".format(i,score))

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
class PolicyNetwork(nn.Module):
    def __init__(self, lr, input_dims, n_actions):
      super(PolicyNetwork, self).__init__()
      self.fc1 = nn.Linear(*input_dims,128)

In [ ]:
optimizer = nn.optimizer